In [8]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import to_categorical
from sklearn import metrics
import numpy as np
import itertools
import matplotlib.pyplot as plt

In [2]:
def plot_confusion_matrix(cm, classes, normalize=False, 
                          title="Matriz de Confusão", 
                          cmap=plt.cm.Blues):
  plt.rcParams["axes.grid"] = False #esconde as linhas do grid
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)
  
  
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalizada")
  else:
    print("Sem normalização")
   
  print(cm)
  
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], 
             horizontalalignment="center", 
             color="white" if cm[i, j] > thresh else "black")
  
  plt.tight_layout()
  plt.ylabel("Classes reais")
  plt.xlabel("Classes previstas")

In [22]:
train_path = "../bases/split1/train"
valid_path = "../bases/split1/valid"
test_path = "../bases/split1/test"
x_size = 767
y_size = 576

# Fine-tunning the VGG16 model

In [30]:
model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(x_size, y_size, 3))

model_vgg.summary()
# Freeze the layers except the last 4 layers
for layer in model_vgg.layers[:-4]:
    layer.trainable = False

model = Sequential()
model.add(model_vgg)

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

#model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 767, 576, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 767, 576, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 767, 576, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 383, 288, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 383, 288, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 383, 288, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 191, 144, 128)     0         
__________

In [25]:
train_batchsize = 8
val_batchsize = 4

train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(x_size, y_size),
        batch_size=train_batchsize,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        valid_path,
        target_size=(x_size, y_size),
        batch_size=val_batchsize,
        class_mode='binary',
        shuffle=False)

Found 192 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [27]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

history = model.fit_generator(train_generator, steps_per_epoch=train_generator.samples/train_generator.batch_size, 
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.samples/validation_generator.batch_size, 
                    epochs=6)


Epoch 1/6


ValueError: Error when checking target: expected dense_5 to have shape (2,) but got array with shape (1,)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
teste_img, test_labels = next(test_generator)

test_labels = test_labels[:,0]

predict = model.predict_generator(test_generator, steps=1, verbose=0)

print(metrics.accuracy_score(test_labels, np.round(predict[:,0])))

print(metrics.classification_report(test_labels, np.round(predict[:,0])))

confusion = metrics.confusion_matrix(test_labels, np.round(predict[:,0]))
print()
print()
cm_plot_labels = ["melanomas", "normais"]

plot_confusion_matrix(confusion, cm_plot_labels, title="Matriz de Confusão")


In [ ]:
model.save("test1.h5")
print("Saved model to disk")